##0. Importing torch and setting up Agnostic code

In [1]:
import torch
from torch import nn

print(torch.__version__)

2.4.1+cu121


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device:{device}')

Device:cpu


In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
